In [1]:
import json
import requests
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import geocoder # import geocoder
import folium
from folium.features import DivIcon

In [2]:
# !conda install geopandas --yes
# !conda install -c conda-forge geocoder
# !conda install -c conda-forge folium --yes 

### Foursquare Setups

In [ ]:
# The code was removed by Watson Studio for sharing.

## Geocoder to get coordinate for the point of interests

In [4]:
def get_latlng(keyword):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      ag = geocoder.arcgis(f'{keyword}, Kuala Lumpur')
      lat_lng_coords = ag.latlng
    
    return lat_lng_coords

## Places of interest as listed by http://www.kuala-lumpur.ws/attractions/10-things-to-see.htm

In [5]:
pois = ["Petronas Twin Tower","Menara Kuala Lumpur","Chinatown","Batu Caves Temple","Sultan Abdul Samad Building","Sunway Lagoon Theme Park","Jalan Alor","Aquaria KLCC","Central Market","Perdana Botanical Gardens"]

## Get the coordinates

In [6]:
thelist = []
for poi in pois:
    lat,long = get_latlng(poi)
    thelist.append((poi,lat,long))

In [7]:
kl_poi_df = pd.DataFrame(thelist,columns=["Place of Interest","Latitude","Longitude"])
kl_poi_df

,Place of Interest,Latitude,Longitude
0,Petronas Twin Tower,3.15832,101.71123
1,Menara Kuala Lumpur,3.15265,101.70332
2,Chinatown,3.14526,101.69746
3,Batu Caves Temple,3.23584,101.68242
4,Sultan Abdul Samad Building,3.14871,101.69422
5,Sunway Lagoon Theme Park,3.07343,101.60707
6,Jalan Alor,3.14526,101.70822
7,Aquaria KLCC,3.15420,101.71348
8,Central Market,3.14432,101.69542
9,Perdana Botanical Gardens,3.14529,101.68314


## Extract nearby venues to Place of interest

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={radius}&limit={limit}"
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['shortName']) for v in results])
        print(f"{name} venues obtained.")

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['POI', 
                  'POI Latitude', 
                  'POI Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Category Alt']
    print("All venues completed")
    return(nearby_venues)

In [9]:
kl_poi_venues = getNearbyVenues(names=kl_poi_df['Place of Interest'],
                                   latitudes=kl_poi_df['Latitude'],
                                   longitudes=kl_poi_df['Longitude']
                                  )

Petronas Twin Tower venues obtained.
Menara Kuala Lumpur venues obtained.
Chinatown venues obtained.
Batu Caves Temple venues obtained.
Sultan Abdul Samad Building venues obtained.
Sunway Lagoon Theme Park venues obtained.
Jalan Alor venues obtained.
Aquaria KLCC venues obtained.
Central Market venues obtained.
Perdana Botanical Gardens venues obtained.
All venues completed


In [10]:
kl_poi_venues.head()

,POI,POI Latitude,POI Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Category Alt
0,Petronas Twin Tower,3.15832,101.71123,Kinokuniya,3.157492,101.712191,Bookstore,Bookstore
1,Petronas Twin Tower,3.15832,101.71123,PETRONAS Twin Towers,3.158436,101.711536,Building,Building
2,Petronas Twin Tower,3.15832,101.71123,W Kuala Lumpur,3.158465,101.709490,Hotel,Hotel
3,Petronas Twin Tower,3.15832,101.71123,Tiffany & Co.,3.157424,101.712290,Jewelry Store,Jewelry
4,Petronas Twin Tower,3.15832,101.71123,SEPHORA,3.157385,101.712270,Cosmetics Shop,Cosmetics


In [11]:
kl_poi_venues['Venue Category Alt'].unique()

array(['Bookstore', 'Building', 'Hotel', 'Jewelry', 'Cosmetics', 'Mall',
       'Cocktail', "Women's Store", 'Boutique', 'Resort', 'Desserts',
       'Art Gallery', 'Concert Hall', 'Science Museum', 'Apparel',
       'Cupcakes', 'Ice Cream', 'Juice Bar', 'Shop', 'Supermarket',
       'Park', 'Snacks', 'Plaza', 'Wine Bar', 'Tex-Mex', 'Coffee Shop',
       'Tapas', 'Spa', 'Fountain', 'Fast Food', 'Candy Store',
       'Kids Store', 'Camera Store', 'Bakery', 'Sushi', 'Bridge',
       'Pharmacy', 'Asian', 'Fried Chicken', 'Breakfast', 'Electronics',
       'Sandwiches', 'Theme Park', 'Department Store', 'Café', 'Lounge',
       'Brazilian', 'Dim Sum', 'Hotel Bar', 'Gym / Fitness', 'Steakhouse',
       'Arts & Crafts', 'Portuguese', 'Pool', 'Market', 'Middle Eastern',
       'Tea Room', 'Persian', 'Bar', 'Halal', 'Landmark',
       'Scenic Lookout', 'Latin American', 'Event Space', 'Italian',
       'Seafood', 'Convenience Store', 'Salad', 'Korean', 'Fabric Shop',
       'Chinese', 'Preserv

## Manually selecting categories to be considered (Food, Bar, and hotel)

In [12]:
selected_categories = 'Hotel','Cocktail','Desserts','Cupcakes','Ice Cream','Juice Bar','Snacks','Wine Bar','Tex-Mex','Coffee Shop','Tapas','Fast Food','Candy Store','Bakery','Sushi','Asian','Fried Chicken','Breakfast','Sandwiches','Café','Lounge','Brazilian','Dim Sum','Hotel Bar','Steakhouse','Portuguese','Cantonese','Middle Eastern','Tea Room','Persian','Bar','Latin American','Italian','Seafood','Salad','Korean','Beer Bar','Pizza','Japanese','Donuts','Malay','Noodles','Speakeasy','Restaurant','Food Truck','Hostel','Chinese','South Indian','Indian','Chettinad','Bubble Tea','Food Court','Thai','Vegetarian / Vegan','Food Stand','B & B','Club','Nightlife','Deli / Bodega','German','Hotpot','Yogurt','Buffet','Burgers','Hookah Bar','Street Food Gathering','Indonesian','BBQ','Hainan','Nightclub','Spanish','French','Halal','Vietnamese','Udon','Whisky Bar','Lebanese','Yakitori','Gourmet','Ramen','Japanese Curry','Food','Gastropub','Food & Drink'

In [13]:
food_restaurant = 'Desserts','Cupcakes','Ice Cream','Snacks','Tex-Mex','Tapas','Fast Food','Candy Store','Bakery','Sushi','Asian','Fried Chicken','Breakfast','Sandwiches','Brazilian','Dim Sum','Steakhouse','Portuguese','Cantonese','Middle Eastern','Persian','Latin American','Italian','Seafood','Salad','Korean','Pizza','Japanese','Donuts','Malay','Noodles','Speakeasy','Restaurant','Food Truck','Chinese','South Indian','Indian','Chettinad','Food Court','Thai','Vegetarian / Vegan','Food Stand','Deli / Bodega','German','Hotpot','Yogurt','Buffet','Burgers','Street Food Gathering','Indonesian','BBQ','Hainan','Spanish','French','Halal','Vietnamese','Udon','Lebanese','Yakitori','Gourmet','Ramen','Japanese Curry','Food','Gastropub','Food & Drink'
bar_cafe = 'Juice Bar','Cocktail','Wine Bar','Coffee Shop','Café','Lounge','Hotel Bar','Tea Room','Bar','Beer Bar','Bubble Tea','Club','Hookah Bar','Nightclub','Nightlife','Whisky Bar'
accomodation = 'Hotel','Hostel','B & B'

## Filter venues to selected categories

In [14]:
kl_poi_venues_filtered = kl_poi_venues[kl_poi_venues['Venue Category Alt'].isin(selected_categories)]

In [15]:
kl_poi_venues_filtered.head()

,POI,POI Latitude,POI Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Category Alt
2,Petronas Twin Tower,3.15832,101.71123,W Kuala Lumpur,3.158465,101.709490,Hotel,Hotel
6,Petronas Twin Tower,3.15832,101.71123,Marini's on 57,3.156738,101.711409,Cocktail Bar,Cocktail
10,Petronas Twin Tower,3.15832,101.71123,"Salon Du Chocolat, Suria KLCC",3.157556,101.712074,Dessert Shop,Desserts
15,Petronas Twin Tower,3.15832,101.71123,Bisou Bake Shop,3.157467,101.712181,Cupcake Shop,Cupcakes
16,Petronas Twin Tower,3.15832,101.71123,Inside Scoop,3.158164,101.712606,Ice Cream Shop,Ice Cream


In [16]:
kl_poi_venues_filtered.groupby('POI').count()

,POI Latitude,POI Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Category Alt
POI,,,,,,,
Aquaria KLCC,57,57,57,57,57,57,57
Batu Caves Temple,6,6,6,6,6,6,6
Central Market,77,77,77,77,77,77,77
Chinatown,51,51,51,51,51,51,51
Jalan Alor,85,85,85,85,85,85,85
Menara Kuala Lumpur,29,29,29,29,29,29,29
Perdana Botanical Gardens,17,17,17,17,17,17,17
Petronas Twin Tower,53,53,53,53,53,53,53
Sultan Abdul Samad Building,67,67,67,67,67,67,67


## Assigning Classes

In [17]:
# kl_poi_venues_filtered[kl_poi_venues_filtered['Venue Category Alt'].isin(accomodation)]['Class'] = "Accomodation"
kl_poi_venues_filtered.loc[kl_poi_venues_filtered['Venue Category Alt'].isin(accomodation),'Class'] = "Accomodation"
kl_poi_venues_filtered.loc[kl_poi_venues_filtered['Venue Category Alt'].isin(bar_cafe),'Class'] = "Bar_Cafe"
kl_poi_venues_filtered.loc[kl_poi_venues_filtered['Venue Category Alt'].isin(food_restaurant),'Class'] = "Food_Restaurant"

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
kl_poi_venues_filtered.head()

,POI,POI Latitude,POI Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Category Alt,Class
2,Petronas Twin Tower,3.15832,101.71123,W Kuala Lumpur,3.158465,101.709490,Hotel,Hotel,Accomodation
6,Petronas Twin Tower,3.15832,101.71123,Marini's on 57,3.156738,101.711409,Cocktail Bar,Cocktail,Bar_Cafe
10,Petronas Twin Tower,3.15832,101.71123,"Salon Du Chocolat, Suria KLCC",3.157556,101.712074,Dessert Shop,Desserts,Food_Restaurant
15,Petronas Twin Tower,3.15832,101.71123,Bisou Bake Shop,3.157467,101.712181,Cupcake Shop,Cupcakes,Food_Restaurant
16,Petronas Twin Tower,3.15832,101.71123,Inside Scoop,3.158164,101.712606,Ice Cream Shop,Ice Cream,Food_Restaurant


### Count of every classes at every POI

In [19]:
kldf01 = kl_poi_venues_filtered[['POI','Class','Venue']].groupby(['POI','Class']).count().sort_values(['POI','Class','Venue'],ascending=False)
kldf01

Venue
POI                         Class                 
Sunway Lagoon Theme Park    Food_Restaurant     37
                            Bar_Cafe            12
                            Accomodation         1
Sultan Abdul Samad Building Food_Restaurant     46
                            Bar_Cafe            14
                            Accomodation         7
Petronas Twin Tower         Food_Restaurant     28
                            Bar_Cafe            19
                            Accomodation         6
Perdana Botanical Gardens   Food_Restaurant     12
                            Bar_Cafe             4
                            Accomodation         1
Menara Kuala Lumpur         Food_Restaurant     15
                            Bar_Cafe             8
                            Accomodation         6
Jalan Alor                  Food_Restaurant     52
                            Bar_Cafe            22
                            Accomodation        11
Chinatown                   Food_Restaurant     29
                            Bar_Cafe            19
                            Accomodation         3
Central Market              Food_Restaurant     48
                            Bar_Cafe            21
                            Accomodation         8
Batu Caves Temple           Food_Restaurant      5
                            Bar_Cafe             1
Aquaria KLCC                Food_Restaurant     26
                            Bar_Cafe            21
                            Accomodation        10

Each places has consistently have more Food Restaurant then bar then accomodation

In [20]:
kl_poi_classes = kldf01.reset_index().pivot(index='POI', columns='Class', values='Venue').reset_index()
kl_poi_classes

Class,POI,Accomodation,Bar_Cafe,Food_Restaurant
0,Aquaria KLCC,10.0,21.0,26.0
1,Batu Caves Temple,NaN,1.0,5.0
2,Central Market,8.0,21.0,48.0
3,Chinatown,3.0,19.0,29.0
4,Jalan Alor,11.0,22.0,52.0
5,Menara Kuala Lumpur,6.0,8.0,15.0
6,Perdana Botanical Gardens,1.0,4.0,12.0
7,Petronas Twin Tower,6.0,19.0,28.0
8,Sultan Abdul Samad Building,7.0,14.0,46.0
9,Sunway Lagoon Theme Park,1.0,12.0,37.0


In [21]:
kl_poi_df

,Place of Interest,Latitude,Longitude
0,Petronas Twin Tower,3.15832,101.71123
1,Menara Kuala Lumpur,3.15265,101.70332
2,Chinatown,3.14526,101.69746
3,Batu Caves Temple,3.23584,101.68242
4,Sultan Abdul Samad Building,3.14871,101.69422
5,Sunway Lagoon Theme Park,3.07343,101.60707
6,Jalan Alor,3.14526,101.70822
7,Aquaria KLCC,3.15420,101.71348
8,Central Market,3.14432,101.69542
9,Perdana Botanical Gardens,3.14529,101.68314


In [22]:
kl_poi_classes_df = kl_poi_df.merge(kl_poi_classes, left_on='Place of Interest', right_on='POI').drop('POI',axis=1).fillna(0)
kl_poi_classes_df

,Place of Interest,Latitude,Longitude,Accomodation,Bar_Cafe,Food_Restaurant
0,Petronas Twin Tower,3.15832,101.71123,6.0,19.0,28.0
1,Menara Kuala Lumpur,3.15265,101.70332,6.0,8.0,15.0
2,Chinatown,3.14526,101.69746,3.0,19.0,29.0
3,Batu Caves Temple,3.23584,101.68242,0.0,1.0,5.0
4,Sultan Abdul Samad Building,3.14871,101.69422,7.0,14.0,46.0
5,Sunway Lagoon Theme Park,3.07343,101.60707,1.0,12.0,37.0
6,Jalan Alor,3.14526,101.70822,11.0,22.0,52.0
7,Aquaria KLCC,3.15420,101.71348,10.0,21.0,26.0
8,Central Market,3.14432,101.69542,8.0,21.0,48.0
9,Perdana Botanical Gardens,3.14529,101.68314,1.0,4.0,12.0


## Break down top venues category for each classes

In [23]:
kldf02 = kl_poi_venues_filtered[['POI','Class','Venue Category Alt','Venue']] \
    .groupby(['POI','Class','Venue Category Alt']).count()

In [24]:
kldf03 = kldf02.sort_values(['POI','Class','Venue','Venue Category Alt'],ascending=False)
kldf03

Venue
POI                         Class           Venue Category Alt          
Sunway Lagoon Theme Park    Food_Restaurant Chinese                    6
                                            Ice Cream                  5
                                            Malay                      3
                                            Asian                      3
                                            Thai                       2
                                            Restaurant                 2
                                            Pizza                      2
                                            Indian                     2
                                            German                     2
                                            Bakery                     2
                                            Yogurt                     1
                                            Sushi                      1
                                            Indonesian                 1
                                            Hotpot                     1
                                            Fried Chicken              1
                                            Fast Food                  1
                                            Burgers                    1
                                            Buffet                     1
                            Bar_Cafe        Coffee Shop                4
                                            Hookah Bar                 2
                                            Café                       2
                                            Bubble Tea                 2
                                            Wine Bar                   1
                                            Juice Bar                  1
                            Accomodation    Hotel                      1
Sultan Abdul Samad Building Food_Restaurant Indian                     9
                                            Malay                      4
                                            Asian                      4
                                            South Indian               3
                                            Food Truck                 3
                                            Food Court                 3
                                            Chinese                    3
                                            Speakeasy                  2
                                            Restaurant                 2
                                            Desserts                   2
                                            Bakery                     2
                                            Vegetarian / Vegan         1
                                            Sandwiches                 1
                                            Noodles                    1
                                            Fried Chicken              1
                                            Food Stand                 1
                                            Food                       1
                                            Deli / Bodega              1
                                            Cupcakes                   1
                                            Chettinad                  1
                            Bar_Cafe        Coffee Shop                5
                                            Café                       4
                                            Wine Bar                   1
                                            Nightlife                  1
                                            Club                       1
                                            Bubble Tea                 1
                                            Bar                        1
                            Accomodation    Hotel                      4
                                            Hostel  

## Get top 5 for food Restaurant Class

In [25]:
for p in pois:
    print('\n',p,'\n', kldf03.loc[p].loc['Food_Restaurant'].head(5))


 Petronas Twin Tower 
                     Venue
Venue Category Alt       
Snacks                  3
Ice Cream               3
Middle Eastern          2
Bakery                  2
Asian                   2

 Menara Kuala Lumpur 
                     Venue
Venue Category Alt       
Sandwiches              2
Latin American          2
Italian                 2
Seafood                 1
Salad                   1

 Chinatown 
                     Venue
Venue Category Alt       
Restaurant              3
Noodles                 3
Indian                  3
Asian                   3
Speakeasy               2

 Batu Caves Temple 
                     Venue
Venue Category Alt       
Asian                   2
Thai                    1
Malay                   1
Indian                  1

 Sultan Abdul Samad Building 
                     Venue
Venue Category Alt       
Indian                  9
Malay                   4
Asian                   4
South Indian            3
Food Truck              3


## Get top 5 for Bar_Cafe Class

In [26]:
for p in pois:
    print('\n',p,'\n', kldf03.loc[p].loc['Bar_Cafe'].head(5))


 Petronas Twin Tower 
                     Venue
Venue Category Alt       
Coffee Shop             5
Juice Bar               3
Café                    3
Lounge                  2
Cocktail                2

 Menara Kuala Lumpur 
                     Venue
Venue Category Alt       
Café                    3
Bar                     2
Coffee Shop             1
Cocktail                1
Beer Bar                1

 Chinatown 
                     Venue
Venue Category Alt       
Café                   12
Coffee Shop             3
Juice Bar               1
Cocktail                1
Bubble Tea              1

 Batu Caves Temple 
                     Venue
Venue Category Alt       
Coffee Shop             1

 Sultan Abdul Samad Building 
                     Venue
Venue Category Alt       
Coffee Shop             5
Café                    4
Wine Bar                1
Nightlife               1
Club                    1

 Sunway Lagoon Theme Park 
                     Venue
Venue Category Alt     

## Visualisation of Results

### Location of POIs

In [27]:
# create map of Kuala Lumpur Places of interest using latitude and longitude values
lat,long = get_latlng("Kuala Lumpur")
kl_poi_df_map = folium.Map(location=[lat, long], zoom_start=12,tiles='cartodbpositron')

# add markers to map
for lat, lng, label in zip(kl_poi_df['Latitude'], kl_poi_df['Longitude'], kl_poi_df['Place of Interest']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(kl_poi_df_map)  
    
kl_poi_df_map

In [28]:
bc = kl_poi_df[kl_poi_df["Place of Interest"] == 'Batu Caves Temple']

In [29]:
kl_poi_classes_df

,Place of Interest,Latitude,Longitude,Accomodation,Bar_Cafe,Food_Restaurant
0,Petronas Twin Tower,3.15832,101.71123,6.0,19.0,28.0
1,Menara Kuala Lumpur,3.15265,101.70332,6.0,8.0,15.0
2,Chinatown,3.14526,101.69746,3.0,19.0,29.0
3,Batu Caves Temple,3.23584,101.68242,0.0,1.0,5.0
4,Sultan Abdul Samad Building,3.14871,101.69422,7.0,14.0,46.0
5,Sunway Lagoon Theme Park,3.07343,101.60707,1.0,12.0,37.0
6,Jalan Alor,3.14526,101.70822,11.0,22.0,52.0
7,Aquaria KLCC,3.15420,101.71348,10.0,21.0,26.0
8,Central Market,3.14432,101.69542,8.0,21.0,48.0
9,Perdana Botanical Gardens,3.14529,101.68314,1.0,4.0,12.0


## Venues count based on classes

In [30]:
# create map of Kuala Lumpur Places of interest using latitude and longitude values
lat,long = get_latlng("Kuala Lumpur")
kl_poi_df_map = folium.Map(location=[lat, long], zoom_start=12,tiles='cartodbpositron')

# add markers to map
for lat, lng, label,fr,br,ht in zip(kl_poi_classes_df['Latitude'], kl_poi_classes_df['Longitude'], kl_poi_classes_df['Place of Interest'], kl_poi_classes_df['Food_Restaurant'], kl_poi_classes_df['Bar_Cafe'], kl_poi_classes_df['Accomodation']):
    label_pop = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label_pop,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(kl_poi_df_map)
    
    folium.map.Marker(
        [lat, lng],
        icon = DivIcon(
            icon_size=(200,36),
            icon_anchor=(0,0),
            html=f"""<div style="font-size: 10pt"><p><strong>{label}</strong><br>
Food:{int(fr)}<br>
Bar:{int(br)}<br>
Hotel:{int(ht)}</p></div>""")
    ).add_to(kl_poi_df_map)
    
    folium.Circle(
        [lat, lng],
        radius=500
    ).add_to(kl_poi_df_map)
    
kl_poi_df_map

## Venues count for each category for class Food/ Restaurant

In [31]:
# create map of Kuala Lumpur Places of interest using latitude and longitude values
lat,long = get_latlng("Kuala Lumpur")
kl_poi_df_map = folium.Map(location=[lat, long], zoom_start=12,tiles='cartodbpositron')

# add markers to map
for lat, lng, label,fr,br,ht in zip(kl_poi_classes_df['Latitude'], kl_poi_classes_df['Longitude'], kl_poi_classes_df['Place of Interest'], kl_poi_classes_df['Food_Restaurant'], kl_poi_classes_df['Bar_Cafe'], kl_poi_classes_df['Accomodation']):
    adf = kldf03.loc[label].loc['Food_Restaurant'].head(5).reset_index()
    adf.columns = ['Category','Count']
    word = adf.to_string(index=False).replace('\n','<br>')
    label_pop = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label_pop,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(kl_poi_df_map)
    
    folium.map.Marker(
        [lat, lng],
        icon = DivIcon(
            icon_size=(200,36),
            icon_anchor=(0,0),
            html=f"""<div style="font-size: 10pt"><strong>{label}</strong><br>
{word}
</div>""")
    ).add_to(kl_poi_df_map)
    
    folium.Circle(
        [lat, lng],
        radius=500
    ).add_to(kl_poi_df_map)
    
kl_poi_df_map

## Venues count for each category for class Bar/ Cafe

In [32]:
# create map of Kuala Lumpur Places of interest using latitude and longitude values
lat,long = get_latlng("Kuala Lumpur")
kl_poi_df_map = folium.Map(location=[lat, long], zoom_start=12,tiles='cartodbpositron')

# add markers to map
for lat, lng, label,fr,br,ht in zip(kl_poi_classes_df['Latitude'], kl_poi_classes_df['Longitude'], kl_poi_classes_df['Place of Interest'], kl_poi_classes_df['Food_Restaurant'], kl_poi_classes_df['Bar_Cafe'], kl_poi_classes_df['Accomodation']):
    adf = kldf03.loc[label].loc['Bar_Cafe'].head(5).reset_index()
    adf.columns = ['Category','Count']
    word = adf.to_string(index=False).replace('\n','<br>')
    label_pop = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label_pop,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(kl_poi_df_map)
    
    folium.map.Marker(
        [lat, lng],
        icon = DivIcon(
            icon_size=(200,36),
            icon_anchor=(0,0),
            html=f"""<div style="font-size: 10pt"><strong>{label}</strong><br>
{word}
</div>""")
    ).add_to(kl_poi_df_map)
    
    folium.Circle(
        [lat, lng],
        radius=500
    ).add_to(kl_poi_df_map)
    
kl_poi_df_map